<div style="padding:20px;color:white;margin:0;font-size:175%;text-align:center;display:fill;border-radius:5px;background-color:#016CC9;overflow:hidden;font-weight:500">Data Wrangling With Pyspark | Pandas | Sql</div>

<h3> Work In Progress</h3>

* Data wrangling using Pandas Dataframe and Pyspark Dataframe ( also Spark Sql)
* Basic steps in each method (structure check)
* Functions to be included 
    * Grouping 
    * Aggregations
    * Windows functions and Other Analytics Function
    * etc.

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%;text-align: center'>1 | Environment Setup</div></b>

### <b><span style='color:#DEB078'>1.1 |</span><span style='color:#016CC9'> Install Pyspark</span></b>  

In [1]:
from IPython.display import clear_output
#!pip install pyspark
clear_output()

### <b><span style='color:#DEB078'>1.2 |</span><span style='color:#016CC9'> Import Required Library </span></b>  

In [2]:
from pyspark.sql import SparkSession
import numpy as np 
import pandas as pd 
import warnings
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### <b><span style='color:#DEB078'>1.3 |</span><span style='color:#016CC9'> Configure Spark Session and Properties </span></b>  

In [3]:
%matplotlib inline
warnings.filterwarnings("ignore")

spark = SparkSession.builder \
          .master("local[*]") \
          .appName("Facts Analysis") \
          .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
clear_output()
spark

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%;text-align: center'>2 | Dataset Setup</div></b>

### <b><span style='color:#DEB078'>2.1 |</span><span style='color:#016CC9'> Creating Pandas Dataframe </span></b>  

pandas_df_15=pd.read_csv("/kaggle/input/world-happiness/2015.csv")
pandas_df_16=pd.read_csv("/kaggle/input/world-happiness/2016.csv")
pandas_df_17=pd.read_csv("/kaggle/input/world-happiness/2017.csv")
pandas_df_18=pd.read_csv("/kaggle/input/world-happiness/2018.csv")
pandas_df_19=pd.read_csv("/kaggle/input/world-happiness/2019.csv")
pandas_df_titanic=pd.read_csv("/kaggle/input/titanic/train.csv")

In [4]:
pandas_df_15=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2015.csv")
pandas_df_16=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2016.csv")
pandas_df_17=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2017.csv")
pandas_df_18=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2018.csv")
pandas_df_19=pd.read_csv("/Users/abhi011097/Workspace/Dataset_Analysis/4_World_Happiness_Report/Dataset/2019.csv")
pandas_df_titanic=pd.read_csv("train-2.csv")

### <b><span style='color:#DEB078'>2.2 |</span><span style='color:#016CC9'> Creating Spark Dataframe </span></b>  

In [5]:
# From Pandas Dataframe
spark_df_15=spark.createDataFrame(pandas_df_15)
spark_df_16=spark.createDataFrame(pandas_df_16)
spark_df_17=spark.createDataFrame(pandas_df_17)
spark_df_18=spark.createDataFrame(pandas_df_18)
spark_df_19=spark.createDataFrame(pandas_df_19)

# From file directly
spark_df_titanic=spark.read\
                      .options(header="true",inferSchema='True')\
                      .csv('train-2.csv')

### <b><span style='color:#DEB078'>2.3 |</span><span style='color:#016CC9'> Creating Spark Table  </span></b>  

In [6]:
spark_df_15.registerTempTable("spark_tbl_15")
spark_df_16.registerTempTable("spark_tbl_16")
spark_df_17.registerTempTable("spark_tbl_17")
spark_df_18.registerTempTable("spark_tbl_18")
spark_df_19.registerTempTable("spark_tbl_19")
spark_df_titanic.registerTempTable("spark_tbl_titanic")

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%;text-align: center'>3 | Investigate Data</div></b>

### <b><span style='color:#DEB078'>3.1 |</span><span style='color:#016CC9'> About Partitions of Spark Dataframe  </span></b>  

In [7]:
spark_df_15.rdd.getNumPartitions() # 8 in my local system as it has 8 cores

8

### <b><span style='color:#DEB078'>3.2 |</span><span style='color:#016CC9'> Dataframe Types  </span></b>  

In [8]:
type(pandas_df_15)

pandas.core.frame.DataFrame

In [9]:
type(spark_df_15)

pyspark.sql.dataframe.DataFrame

### <b><span style='color:#DEB078'>3.3 |</span><span style='color:#016CC9'> Check Schrema  </span></b>  

#### Using Pandas

In [10]:
pandas_df_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        158 non-null    object 
 1   Region                         158 non-null    object 
 2   Happiness Rank                 158 non-null    int64  
 3   Happiness Score                158 non-null    float64
 4   Standard Error                 158 non-null    float64
 5   Economy (GDP per Capita)       158 non-null    float64
 6   Family                         158 non-null    float64
 7   Health (Life Expectancy)       158 non-null    float64
 8   Freedom                        158 non-null    float64
 9   Trust (Government Corruption)  158 non-null    float64
 10  Generosity                     158 non-null    float64
 11  Dystopia Residual              158 non-null    float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1

#### Using Spark Dataframe

In [11]:
spark_df_15.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Happiness Rank: long (nullable = true)
 |-- Happiness Score: double (nullable = true)
 |-- Standard Error: double (nullable = true)
 |-- Economy (GDP per Capita): double (nullable = true)
 |-- Family: double (nullable = true)
 |-- Health (Life Expectancy): double (nullable = true)
 |-- Freedom: double (nullable = true)
 |-- Trust (Government Corruption): double (nullable = true)
 |-- Generosity: double (nullable = true)
 |-- Dystopia Residual: double (nullable = true)



### <b><span style='color:#DEB078'>3.4 |</span><span style='color:#016CC9'> Describe Dataframe  </span></b>  

#### For Spark Dataframe

In [12]:
spark_df_15.describe().show(truncate=False)

+-------+-----------+-------------------------+-----------------+------------------+--------------------+------------------------+------------------+------------------------+-------------------+-----------------------------+-------------------+------------------+
|summary|Country    |Region                   |Happiness Rank   |Happiness Score   |Standard Error      |Economy (GDP per Capita)|Family            |Health (Life Expectancy)|Freedom            |Trust (Government Corruption)|Generosity         |Dystopia Residual |
+-------+-----------+-------------------------+-----------------+------------------+--------------------+------------------------+------------------+------------------------+-------------------+-----------------------------+-------------------+------------------+
|count  |158        |158                      |158              |158               |158                 |158                     |158               |158                     |158                |158           

#### For Pandas Dataframe

In [13]:
pandas_df_15.describe()

,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
count,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000,158.000000
mean,79.493671,5.375734,0.047885,0.846137,0.991046,0.630259,0.428615,0.143422,0.237296,2.098977
std,45.754363,1.145010,0.017146,0.403121,0.272369,0.247078,0.150693,0.120034,0.126685,0.553550
min,1.000000,2.839000,0.018480,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.328580
25%,40.250000,4.526000,0.037268,0.545808,0.856823,0.439185,0.328330,0.061675,0.150553,1.759410
50%,79.500000,5.232500,0.043940,0.910245,1.029510,0.696705,0.435515,0.107220,0.216130,2.095415
75%,118.750000,6.243750,0.052300,1.158448,1.214405,0.811013,0.549092,0.180255,0.309883,2.462415
max,158.000000,7.587000,0.136930,1.690420,1.402230,1.025250,0.669730,0.551910,0.795880,3.602140


In [14]:
pandas_df_15.describe(include='O')

,Country,Region
count,158,158
unique,158,10
top,Switzerland,Sub-Saharan Africa
freq,1,40


## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%;text-align: center'>4 | Basic Introduction with Titanic Data</div></b>

### <b><span style='color:#DEB078'>4.1 |</span><span style='color:#016CC9'> Select | Alias | Limit  Data </span></b>  
 
* Select -> To pick or filter columns  (All features or few features)
* Alias  ->  Temporary name for columns , table etc.
* Limit  ->  Limit can be used to restrict number of rows (Unordered)

#### Using Spark Sql 

In [17]:
# For all columns use astrick * & limit followed by number for no of rows to be returned if available
print ("All columns  and 2 rows only")

query="""Select * 
             From spark_tbl_titanic  
             Limit 2 """

spark.sql(query).show()


# For columns to be viewed provide column name instead of * 
print("Selected columns to be viewed along with Alias on column")

query="""Select Name, 
             Pclass as PassengerClass 
             From spark_tbl_titanic 
             Limit 2"""

spark.sql(query).show()  # To select column with space use backticks

All columns  and 2 rows only
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+

Selected columns to be viewed along with Alias on column
+--------------------+--------------+
|                Name|PassengerClass|
+--------------------+--------------+
|Braund, Mr. Owen ...|             3|
|Cumings, Mrs. Joh...|             1|
+--------------------+--------------+



#### Using Pandas Dataframe

In [18]:
 # For all columns and rows 
    # pandas_df_titanic
    # pandas_df_titanic.iloc[:,:]  --> takes index postitions and column positions   [row,column] 
    # pandas_df_titanic.loc[:,:]  --> takes index & column name 
    
# All columns and 2 rows 
    # pandas_df_titanic.iloc[0:2,:] 
    # pandas_df_titanic.head(2)
    # pandas_df_titanic.loc[0:2,:]
    
pandas_df_titanic.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [19]:
# For Selected column and with alias and 2 rows
pandas_df_titanic.loc[0:1,['PassengerId','Survived']].rename(columns={'PassengerId':'Id'})     

    # pandas_df_titanic.iloc[0:2,[0,1]].rename(columns={'PassengerId':'Id'})                     --> can also provide slicing 
    # pandas_df_titanic[['PassengerId','Survived']].rename(columns={'PassengerId':'Id'}).head(2)

,Id,Survived
0,1,0
1,2,1


#### Using Spark Dataframe

In [20]:
# For all Columns and 2 rows
spark_df_titanic.select("*")\
                .limit(2)\
                .show()
        # spark_df_titanic.limit(2).show()


# For Selected column and with alias and 2 rows
spark_df_titanic.select(
                        F.col("PassengerId").alias('Id'),
                        "Survived"
                        )\
                .limit(2)\
                .show()

                # spark_df_titanic.select("PassengerId","Survived").withColumnRenamed("PassengerId","Id").limit(2).show()
    
# Select  method to select columns list 
# F.col -> to select column and perform operation on it 
# alias method to rename temporarily

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+

+---+--------+
| Id|Survived|
+---+--------+
|  1|       0|
|  2|       1|
+---+--------+



### <b><span style='color:#DEB078'>4.2 | </span><span style='color:#016CC9'> Filter | Order Data </span></b>  
* Filter -> condition based restriction on data to be presented
* Order -> To present data in some order (ascending,descending)

#### Using Spark Sql

In [21]:
# Sorted by fare in descending followed by PassengerId in ascending
# Where is used to filter and order by to provide order

spark.sql("""select * 
             From spark_tbl_titanic 
             Where Survived=1 
             Order by Fare desc, PassengerId asc 
             Limit 4
          """).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+



#### Using Pandas Dataframe

In [22]:
row_mask=(pandas_df_titanic['Survived']==1)

pandas_df_titanic.loc[row_mask,:]\
                                 .sort_values(by=['Fare','PassengerId'],ascending=[False,True])\
                                 .head(4)

# sort_values is used to order data 
# row mask containing true and false passed in rows index (true index will be returned) 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
258,259,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,NaN,C
679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.0,0,1,PC 17755,512.3292,B51 B53 B55,C
737,738,1,1,"Lesurer, Mr. Gustave J",male,35.0,0,0,PC 17755,512.3292,B101,C
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S


### Using Spark Dataframe

In [23]:
spark_df_titanic\
                .filter(F.col('Survived')==1)\
                .sort(['Fare','PassengerId'],ascending=[False,True])\
                .limit(4)\
                .show()

# filter used to provide condition to filter rows
# sort method to order data

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|
|        680|       1|     1|Cardeza, Mr. Thom...|  male|36.0|    0|    1|PC 17755|512.3292|B51 B53 B55|       C|
|        738|       1|     1|Lesurer, Mr. Gust...|  male|35.0|    0|    0|PC 17755|512.3292|       B101|       C|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+



### <b><span style='color:#DEB078'>4.3 | </span><span style='color:#016CC9'> Aggregations & Grouping Data </span></b>  

#### Using Spark Sql

In [24]:
spark.sql("select survived , count(1) from spark_tbl_titanic group by 1 ").show()

+--------+--------+
|survived|count(1)|
+--------+--------+
|       1|     342|
|       0|     549|
+--------+--------+



In [47]:
spark.sql("select survived , min(Age) Age_Min , max(Age) Age_Max ,sum(Fare) Total_Fare from spark_tbl_titanic group by 1 ").show()

+--------+-------+-------+------------------+
|survived|Age_Min|Age_Max|        Total_Fare|
+--------+-------+-------+------------------+
|       1|   0.42|   80.0|16551.229399999997|
|       0|    1.0|   74.0|12142.719899999987|
+--------+-------+-------+------------------+



#### Using Spark Dataframe

In [25]:
spark_df_titanic\
        .groupBy('survived')\
        .count()\
        .show()

+--------+-----+
|survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [52]:
spark_df_titanic\
    .groupBy('survived')\
    .agg(F.min("Age").alias("Age_Min") , 
         F.max("Age").alias("Age_Max") ,
         F.sum("Fare").alias("Total_Fare") ) \
    .show()

+--------+-------+-------+------------------+
|survived|Age_Min|Age_Max|        Total_Fare|
+--------+-------+-------+------------------+
|       1|   0.42|   80.0|16551.229399999997|
|       0|    1.0|   74.0|12142.719899999987|
+--------+-------+-------+------------------+



#### Using Pandas Dataframe

In [53]:
#pandas_df_titanic['Survived'].value_counts() 
# pandas_df_titanic.groupby('Survived')['Survived'].count()
pandas_df_titanic.groupby('Survived').size()

Survived
0    549
1    342
dtype: int64

In [46]:
# Aggregation

pandas_df_titanic.groupby("Survived").agg(
     age_min=pd.NamedAgg(column="Age", aggfunc="min"),
     age_max=pd.NamedAgg(column="Age", aggfunc="max"),
     Fare=pd.NamedAgg(column="Fare", aggfunc="sum"),

)

#pandas_df_titanic.groupby('Survived').agg({'Age': ['min', 'max'], 'Fare': 'sum'})

,age_min,age_max,Fare
Survived,,,
0,1.00,74.0,12142.7199
1,0.42,80.0,16551.2294


### Querying Multiple Datasources (Joins)  --Inprogress

## <b><div style='padding:15px;background-color:#79A7D3;color:white;border-radius:2px;font-size:90%;text-align: center'>5 | Complex Query Using World Happiness Data</div></b>

### <b><span style='color:#DEB078'>5.1 | </span><span style='color:#016CC9'> Understand features available for reporting in all files </span></b>  

In [20]:
print("pandas_df_15 -> ",pandas_df_15.columns.to_list())
print("pandas_df_16 -> ",pandas_df_16.columns.to_list())
print("pandas_df_17 -> ",pandas_df_17.columns.to_list())
print("pandas_df_18 -> ",pandas_df_18.columns.to_list())
print("pandas_df_19 -> ",pandas_df_19.columns.to_list())

pandas_df_15 ->  ['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
pandas_df_16 ->  ['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']
pandas_df_17 ->  ['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']
pandas_df_18 ->  ['Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']
pandas_df_19 ->  ['Overall rank', 'Cou

### <b><span style='color:#DEB078'>5.2 | </span><span style='color:#016CC9'> In 2019 Performance of top 5 Happiest countries in each of category </span></b>  

#### Using Spark Sql

In [21]:
spark.sql("""select `Overall rank` Overall_rank,`Country or region` Country,
          rank() over (order by `GDP per capita` desc) as GDP_per_capita_Rank,
          rank() over (order by `Social support` desc) as Social_Support_Rank,
          rank() over (order by `Healthy life expectancy` desc) as Health_Rank,
          rank() over (order by `Freedom to make life choices` desc) as Freedom_Rank,
          rank() over (order by `Generosity` desc) as Generosity_Rank,
          rank() over (order by `Perceptions of corruption` desc) as Perceptions_corruption_Rank
          from spark_tbl_19
          order by `Overall rank` 
          limit 10
          """).show(5)

+------------+-----------+-------------------+-------------------+-----------+------------+---------------+---------------------------+
|Overall_rank|    Country|GDP_per_capita_Rank|Social_Support_Rank|Health_Rank|Freedom_Rank|Generosity_Rank|Perceptions_corruption_Rank|
+------------+-----------+-------------------+-------------------+-----------+------------+---------------+---------------------------+
|           1|    Finland|                 23|                  2|         28|           5|             92|                          4|
|           2|    Denmark|                 14|                  4|         25|           6|             37|                          3|
|           3|     Norway|                  7|                  3|         14|           3|             22|                          8|
|           4|    Iceland|                 15|                  1|         15|           7|              8|                         49|
|           5|Netherlands|                 12|  

#### Using Pandas

In [22]:
pd.concat([pandas_df_19[['Overall rank','Country or region']],pandas_df_19.rank(
    axis = 0,
    method = 'min',
    numeric_only = True,
    na_option = 'keep',
    ascending = False,
    pct = False
).astype(np.int16).iloc[:,2:]],axis=1).iloc[:5,:]

,Overall rank,Country or region,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,23,2,28,5,92,4
1,2,Denmark,14,4,25,6,37,3
2,3,Norway,7,3,14,3,22,8
3,4,Iceland,15,1,15,7,8,49
4,5,Netherlands,12,15,20,17,13,12


#### Using Spark Dataframe

In [23]:

spark_df_19\
    .drop('Score')\
    .withColumn("GDP per capita",F.rank().over(Window.orderBy(F.col('`GDP per capita`').desc()))) \
    .withColumn("Social support",F.rank().over(Window.orderBy(F.col("Social support").desc()))) \
    .withColumn("Healthy life expectancy",F.rank().over(Window.orderBy(F.col("Healthy life expectancy").desc()))) \
    .withColumn("Freedom to make life choices",F.rank().over(Window.orderBy(F.col("Freedom to make life choices").desc()))) \
    .withColumn("Generosity",F.rank().over(Window.orderBy(F.col("Generosity").desc()))) \
    .withColumn("Perceptions of corruption",F.rank().over(Window.orderBy(F.col("Perceptions of corruption").desc()))) \
    .sort('Overall rank')\
    .show(5)

+------------+-----------------+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|Overall rank|Country or region|GDP per capita|Social support|Healthy life expectancy|Freedom to make life choices|Generosity|Perceptions of corruption|
+------------+-----------------+--------------+--------------+-----------------------+----------------------------+----------+-------------------------+
|           1|          Finland|            23|             2|                     28|                           5|        92|                        4|
|           2|          Denmark|            14|             4|                     25|                           6|        37|                        3|
|           3|           Norway|             7|             3|                     14|                           3|        22|                        8|
|           4|          Iceland|            15|             1|                    

### <b><span style='color:#DEB078'>5.3 | </span><span style='color:#016CC9'> Find the top 3 Countries in each year  </span></b>  
* sorted in yearwise followed by rank

* Output Format Using Pivot

    *  2015  | rank1 | rank2 | rank3 | rank4 | rank5
    *  2016  | rank1 | rank2 | rank3 | rank4 | rank5
    * ...
    

#### Using Spark Sql

In [24]:
spark.sql("""
            with ref_tbl as 
                    (Select year,rank,country from (
                        select country,`Happiness Rank` as rank , '2015' year  from spark_tbl_15 where `Happiness Rank`  <= 5
                        union
                        select country,`Happiness Rank` as rank , '2016' year  from spark_tbl_16 where `Happiness Rank`  <= 5
                        union
                        select country,`Happiness.Rank` as rank , '2017' year  from spark_tbl_17 where `Happiness.Rank`  <= 5
                        union
                        select `Country or region` country,`Overall rank` as rank , '2018' year  from spark_tbl_18 where `Overall rank`  <= 5
                        union
                        select `Country or region` country,`Overall rank` as rank , '2019' year  from spark_tbl_19 where `Overall rank`  <= 5       
                      ) order by 1 asc,2 asc)
                      
            select * from ref_tbl
            PIVOT (
                      max(country)
                      FOR rank in (
                        1 Rank_1, 2 Rank_2, 3 Rank_3, 4 Rank_4, 5 Rank_5
                      )
                    )
            ORDER BY year
                      """).show()

+----+-----------+-----------+-------+-----------+-----------+
|year|     Rank_1|     Rank_2| Rank_3|     Rank_4|     Rank_5|
+----+-----------+-----------+-------+-----------+-----------+
|2015|Switzerland|    Iceland|Denmark|     Norway|     Canada|
|2016|    Denmark|Switzerland|Iceland|     Norway|    Finland|
|2017|     Norway|    Denmark|Iceland|Switzerland|    Finland|
|2018|    Finland|     Norway|Denmark|    Iceland|Switzerland|
|2019|    Finland|    Denmark| Norway|    Iceland|Netherlands|
+----+-----------+-----------+-------+-----------+-----------+



#### Using Pandas

In [25]:
temp_res=pd.concat([
    pandas_df_15[['Country','Happiness Rank']].assign(Year='2015').rename(columns={'Happiness Rank':'Rank'}), 
    pandas_df_16[['Country','Happiness Rank']].assign(Year='2016').rename(columns={'Happiness Rank':'Rank'}),
    pandas_df_17[['Country','Happiness.Rank']].assign(Year='2017').rename(columns={'Happiness.Rank':'Rank'}),
    pandas_df_18[['Country or region','Overall rank']].assign(Year='2018').rename(columns={'Country or region':'Country','Overall rank':'Rank'}),
    pandas_df_19[['Country or region','Overall rank']].assign(Year='2019').rename(columns={'Country or region':'Country','Overall rank':'Rank'})
        ])

temp_res[temp_res['Rank']<=5]\
    .pivot(index='Year', columns='Rank', values='Country')\
    .rename(columns={1:'Rank_1',2:'Rank_2',3:'Rank_3',4:'Rank_4',5:'Rank_5'})\
    .rename_axis(None, axis=1)

,Rank_1,Rank_2,Rank_3,Rank_4,Rank_5
Year,,,,,
2015,Switzerland,Iceland,Denmark,Norway,Canada
2016,Denmark,Switzerland,Iceland,Norway,Finland
2017,Norway,Denmark,Iceland,Switzerland,Finland
2018,Finland,Norway,Denmark,Iceland,Switzerland
2019,Finland,Denmark,Norway,Iceland,Netherlands


#### Using Spark Dataframe

In [26]:
spark_df_15['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank").withColumn('Year',F.lit(2015))\
    .union(spark_df_16['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank").withColumn('Year',F.lit(2016)))\
    .union(spark_df_17.select('Country',F.col('`Happiness.Rank`').alias('Rank'),F.lit(2017).alias('Year')))\
    .union(spark_df_18.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank'),F.lit(2018).alias('Year')))\
    .union(spark_df_19.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank'),F.lit(2019).alias('Year')))\
    .filter(F.col('Rank') <=5)\
    .groupBy("Year").pivot("Rank").agg(F.first('Country'))\
    .withColumnRenamed("1","Rank_1").withColumnRenamed("2","Rank_2").withColumnRenamed("3","Rank_3")\
    .withColumnRenamed("4","Rank_4").withColumnRenamed("5","Rank_5")\
    .show()

+----+-----------+-----------+-------+-----------+-----------+
|Year|     Rank_1|     Rank_2| Rank_3|     Rank_4|     Rank_5|
+----+-----------+-----------+-------+-----------+-----------+
|2015|Switzerland|    Iceland|Denmark|     Norway|     Canada|
|2016|    Denmark|Switzerland|Iceland|     Norway|    Finland|
|2017|     Norway|    Denmark|Iceland|Switzerland|    Finland|
|2018|    Finland|     Norway|Denmark|    Iceland|Switzerland|
|2019|    Finland|    Denmark| Norway|    Iceland|Netherlands|
+----+-----------+-----------+-------+-----------+-----------+



### <b><span style='color:#DEB078'>5.4 | </span><span style='color:#016CC9'> Find the countries which never observed a drop in rank along with year wise rank and diffence of rank from 2015 to 2019 </span></b>   
* Sort the data by biggest to lowest change between 2015 to 2019

#### Using Spark Sql

In [27]:
spark.sql("""Select t1.Country ,t1.`Happiness Rank` as Rank_2015,t2.`Happiness Rank` Rank_2016,t3.`Happiness.Rank`  Rank_2017, 
            t4.`Overall rank` Rank_2018 ,t5.`Overall rank` Rank_2019,
            (t1.`Happiness Rank`-t5.`Overall rank`) Increment_Factor
            from  spark_tbl_15  t1
            inner join spark_tbl_16 t2
            on t1.Country = t2.Country and t1.`Happiness Rank` >= t2.`Happiness Rank`
            inner join spark_tbl_17 t3
            on t1.Country = t3.Country and t2.`Happiness Rank` >= t3.`Happiness.Rank`
            inner join spark_tbl_18 t4
            on t1.Country = t4.`Country or region` and t3.`Happiness.Rank` >= t4.`Overall rank`
            inner join spark_tbl_19 t5
            on t1.Country = t5.`Country or region` and t4.`Overall rank` >= t5.`Overall rank`
            order by Increment_Factor desc
          
          """).show(200)

+-------------------+---------+---------+---------+---------+---------+----------------+
|            Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Increment_Factor|
+-------------------+---------+---------+---------+---------+---------+----------------+
|              Benin|      155|      153|      143|      136|      102|              53|
|        Ivory Coast|      151|      139|      128|      107|       99|              52|
|           Honduras|      105|      104|       91|       72|       59|              46|
|            Hungary|      104|       91|       75|       69|       62|              42|
|            Romania|       86|       71|       57|       52|       48|              38|
|       Burkina Faso|      152|      145|      134|      121|      115|              37|
|           Cameroon|      133|      114|      107|       99|       96|              37|
|           Bulgaria|      134|      129|      105|      100|       97|              37|
|Congo (Brazzaville)|

#### Using Pandas

In [28]:
temp_res=pd.merge(pandas_df_15[['Country','Happiness Rank']], pandas_df_16[['Country','Happiness Rank']], on="Country")\
        .rename(columns={'Happiness Rank_x':'Rank_2015','Happiness Rank_y':'Rank_2016'})\
        .query("Rank_2015 >= Rank_2016")\
        .merge(pandas_df_17[['Country','Happiness.Rank']],on="Country")\
        .rename(columns={'Happiness.Rank':'Rank_2017'})\
        .query("Rank_2016 >= Rank_2017")\
        .merge(pandas_df_18[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2018'})\
        .query("Rank_2017 >= Rank_2018")\
        .drop(columns='Country or region')\
        .merge(pandas_df_19[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2019'})\
        .query("Rank_2018 >= Rank_2019")\
        .drop(columns='Country or region')
        

temp_res.assign(Increment_Factor=temp_res["Rank_2015"] - temp_res["Rank_2019"])\
        .sort_values('Increment_Factor',ascending=False)\
        .style.hide_index()

Country,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Increment_Factor
Benin,155,153,143,136,102,53
Ivory Coast,151,139,128,107,99,52
Honduras,105,104,91,72,59,46
Hungary,104,91,75,69,62,42
Romania,86,71,57,52,48,38
Cameroon,133,114,107,99,96,37
Burkina Faso,152,145,134,121,115,37
Bulgaria,134,129,105,100,97,37
Cambodia,145,140,129,120,109,36
Congo (Brazzaville),139,127,124,114,103,36


#### Using Spark Dataframe

In [29]:
spark_df_15['Country','Happiness Rank']\
    .withColumnRenamed("Happiness Rank","Rank_2015")\
    .join(spark_df_16['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank_2016"),on='Country')\
    .filter(F.col("Rank_2015") >= F.col("Rank_2016"))\
    .join(spark_df_17['Country','`Happiness.Rank`'].withColumnRenamed("Happiness.Rank","Rank_2017"),on='Country')\
    .filter(F.col("Rank_2016") >= F.col("Rank_2017"))\
    .join(spark_df_18.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank_2018')),on='Country')\
    .filter(F.col("Rank_2017") >= F.col("Rank_2018"))\
    .join(spark_df_19.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank_2019')),on='Country')\
    .filter(F.col("Rank_2018") >= F.col("Rank_2019"))\
    .withColumn('Increment_Factor', F.col("Rank_2015") - F.col("Rank_2019"))\
    .sort('Increment_Factor',ascending=False)\
    .show(200)
    

+-------------------+---------+---------+---------+---------+---------+----------------+
|            Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Increment_Factor|
+-------------------+---------+---------+---------+---------+---------+----------------+
|              Benin|      155|      153|      143|      136|      102|              53|
|        Ivory Coast|      151|      139|      128|      107|       99|              52|
|           Honduras|      105|      104|       91|       72|       59|              46|
|            Hungary|      104|       91|       75|       69|       62|              42|
|            Romania|       86|       71|       57|       52|       48|              38|
|       Burkina Faso|      152|      145|      134|      121|      115|              37|
|           Cameroon|      133|      114|      107|       99|       96|              37|
|           Bulgaria|      134|      129|      105|      100|       97|              37|
|Congo (Brazzaville)|

### <b><span style='color:#DEB078'>5.5 | </span><span style='color:#016CC9'> Find the countries which continously observed drop in rank along with year wise rank and diffence of rank from 2015 to 2019  </span></b>   

* Sort the data by biggest to lowest change between 2015 to 2019

#### Using Spark Sql

In [30]:
spark.sql("""Select t1.Country ,t1.`Happiness Rank` as Rank_2015,t2.`Happiness Rank` Rank_2016,t3.`Happiness.Rank`  Rank_2017, 
            t4.`Overall rank` Rank_2018 ,t5.`Overall rank` Rank_2019,
            (t1.`Happiness Rank`-t5.`Overall rank`) Decrement_Factor
            from  spark_tbl_15  t1
            inner join spark_tbl_16 t2
            on t1.Country = t2.Country and t1.`Happiness Rank` < t2.`Happiness Rank`
            inner join spark_tbl_17 t3
            on t1.Country = t3.Country and t2.`Happiness Rank` < t3.`Happiness.Rank`
            inner join spark_tbl_18 t4
            on t1.Country = t4.`Country or region` and t3.`Happiness.Rank` < t4.`Overall rank`
            inner join spark_tbl_19 t5
            on t1.Country = t5.`Country or region` and t4.`Overall rank` < t5.`Overall rank`
            order by Decrement_Factor asc
          
          """).show(200)

+-----------+---------+---------+---------+---------+---------+----------------+
|    Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Decrement_Factor|
+-----------+---------+---------+---------+---------+---------+----------------+
|  Venezuela|       23|       44|       82|      102|      108|             -85|
|     Zambia|       85|      106|      116|      125|      138|             -53|
|   Zimbabwe|      115|      131|      138|      144|      146|             -31|
|      India|      117|      118|      122|      133|      140|             -23|
|    Belarus|       59|       61|       67|       73|       81|             -22|
|   Botswana|      128|      137|      142|      146|      148|             -20|
|     Malawi|      131|      132|      136|      147|      150|             -19|
|    Moldova|       52|       55|       56|       67|       71|             -19|
|     Brazil|       16|       17|       22|       28|       32|             -16|
| Azerbaijan|       80|     

#### Using Pandas

In [31]:

temp_res=pd.merge(pandas_df_15[['Country','Happiness Rank']], pandas_df_16[['Country','Happiness Rank']], on="Country")\
        .rename(columns={'Happiness Rank_x':'Rank_2015','Happiness Rank_y':'Rank_2016'})\
        .query("Rank_2015 < Rank_2016")\
        .merge(pandas_df_17[['Country','Happiness.Rank']],on="Country")\
        .rename(columns={'Happiness.Rank':'Rank_2017'})\
        .query("Rank_2016 < Rank_2017")\
        .merge(pandas_df_18[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2018'})\
        .query("Rank_2017 < Rank_2018")\
        .drop(columns='Country or region')\
        .merge(pandas_df_19[['Country or region','Overall rank']],left_on="Country",right_on="Country or region")\
        .rename(columns={'Overall rank':'Rank_2019'})\
        .query("Rank_2018 < Rank_2019")\
        .drop(columns='Country or region')
        

temp_res.assign(Decrement_Factor=temp_res["Rank_2015"] - temp_res["Rank_2019"])\
        .sort_values('Decrement_Factor')\
        .style.hide_index()

Country,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Decrement_Factor
Venezuela,23,44,82,102,108,-85
Zambia,85,106,116,125,138,-53
Zimbabwe,115,131,138,144,146,-31
India,117,118,122,133,140,-23
Belarus,59,61,67,73,81,-22
Botswana,128,137,142,146,148,-20
Moldova,52,55,56,67,71,-19
Malawi,131,132,136,147,150,-19
Brazil,16,17,22,28,32,-16
Azerbaijan,80,81,85,87,90,-10


#### Using Spark Dataframe

In [32]:
spark_df_15['Country','Happiness Rank']\
    .withColumnRenamed("Happiness Rank","Rank_2015")\
    .join(spark_df_16['Country','Happiness Rank'].withColumnRenamed("Happiness Rank","Rank_2016"),on='Country')\
    .filter(F.col("Rank_2015") < F.col("Rank_2016"))\
    .join(spark_df_17['Country','`Happiness.Rank`'].withColumnRenamed("Happiness.Rank","Rank_2017"),on='Country')\
    .filter(F.col("Rank_2016") < F.col("Rank_2017"))\
    .join(spark_df_18.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank_2018')),on='Country')\
    .filter(F.col("Rank_2017") < F.col("Rank_2018"))\
    .join(spark_df_19.select(F.col('Country or region').alias('Country'),F.col('`Overall rank`').alias('Rank_2019')),on='Country')\
    .filter(F.col("Rank_2018") < F.col("Rank_2019"))\
    .withColumn('Decrement_Factor', F.col("Rank_2015") - F.col("Rank_2019"))\
    .sort('Decrement_Factor')\
    .show(200)
    

+-----------+---------+---------+---------+---------+---------+----------------+
|    Country|Rank_2015|Rank_2016|Rank_2017|Rank_2018|Rank_2019|Decrement_Factor|
+-----------+---------+---------+---------+---------+---------+----------------+
|  Venezuela|       23|       44|       82|      102|      108|             -85|
|     Zambia|       85|      106|      116|      125|      138|             -53|
|   Zimbabwe|      115|      131|      138|      144|      146|             -31|
|      India|      117|      118|      122|      133|      140|             -23|
|    Belarus|       59|       61|       67|       73|       81|             -22|
|   Botswana|      128|      137|      142|      146|      148|             -20|
|     Malawi|      131|      132|      136|      147|      150|             -19|
|    Moldova|       52|       55|       56|       67|       71|             -19|
|     Brazil|       16|       17|       22|       28|       32|             -16|
| Azerbaijan|       80|     

### In progress

In [ ]:
#spark.stop()